## Load the model from the web and save it

In [2]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)
tokenizer.save_pretrained(MODEL)

2023-09-18 20:38:46.810413: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


('cardiffnlp/twitter-xlm-roberta-base-sentiment/tokenizer_config.json',
 'cardiffnlp/twitter-xlm-roberta-base-sentiment/special_tokens_map.json',
 'cardiffnlp/twitter-xlm-roberta-base-sentiment/sentencepiece.bpe.model',
 'cardiffnlp/twitter-xlm-roberta-base-sentiment/added_tokens.json',
 'cardiffnlp/twitter-xlm-roberta-base-sentiment/tokenizer.json')

## Import statements for libraries

In [5]:
import numpy as np
from scipy.special import softmax
import nltk
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('wordnet')
import pandas as pd
import csv

[nltk_data] Downloading package stopwords to /Users/sulu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/sulu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Function to preprocess the text

In [6]:
# Laden Sie den Datensatz mit den Unicode-Namen der Emojis
emoji_data = {}
with open('../../data/emoji_dataset.csv', mode='r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        emoji_data[row[2]] = row[3]

In [7]:
def ersetze_emojis(text):
    for emoji, name in emoji_data.items():
        text = text.replace(emoji, name)
    # Entfernen Sie Leerzeichen am Anfang und am Ende des Texts
    text = text.strip()
    return text

In [8]:
# Preprocess text (username and link placeholders)
def preprocess(text):

    # emojis entfernen
    #text = ersetze_emojis(text)

    # lemmatizer = WordNetLemmatizer()
    # words = text.split()
    # lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    # text = ' '.join(lemmatized_words)


    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

## Test the model on validation data

In [9]:
# Laden Sie Ihren CSV-Datensatz
csv_file = "../../data/val_data.csv"  # Ersetzen Sie durch den Pfad zu Ihrer CSV-Datei
df = pd.read_csv(csv_file)

# Leere Listen zum Speichern der vorhergesagten Sentiments und der tatsächlichen Labels
predicted_sentiments = []
actual_labels = []

count = 0
# Durchlaufen Sie jeden Eintrag im DataFrame und wenden Sie das Modell an
for index, row in df.iterrows():
    text = row['text']
    text = preprocess(text)  # Stellen Sie sicher, dass Sie Ihre Preprocessing-Funktion hier verwenden
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    softmax_scores = softmax(scores)

    # Extrahieren Sie das vorhergesagte Sentiment
    predicted_label_id = np.argmax(softmax_scores)
    #predicted_sentiment = model.config.id2label[predicted_label_id]

    # Speichern Sie das vorhergesagte Sentiment und das tatsächliche Label
    predicted_sentiments.append(predicted_label_id)
    actual_labels.append(row['label'])

# Fügen Sie die vorhergesagten Sentiments und die tatsächlichen Labels als neue Spalten zum DataFrame hinzu
df['predicted_sentiment'] = predicted_sentiments

# Vergleichen Sie die vorhergesagten Sentiments mit den tatsächlichen Labels
correct_predictions = (df['predicted_sentiment'] == df['label']).sum()
total_predictions = len(df)

# Berechnen Sie die Genauigkeit (Accuracy) der Vorhersagen
accuracy = correct_predictions / total_predictions

# Drucken Sie die Genauigkeit
print(f"Genauigkeit: {accuracy:.2%}")

Genauigkeit: 74.05%


## Print out statistics and confusion matrix

In [10]:
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, recall_score

# Ihre Verwirrungsmatrix
conf_matrix = confusion_matrix(df['label'], df['predicted_sentiment'])

# Berechnen Sie die Precision und den Recall für jede Klasse
precision = precision_score(df['label'], df['predicted_sentiment'], average=None)
recall = recall_score(df['label'], df['predicted_sentiment'], average=None)

# Berechnen Sie den gewichteten Durchschnitt von Precision und Recall (Macro-Durchschnitt)
macro_precision = precision_score(df['label'], df['predicted_sentiment'], average='macro')
macro_recall = recall_score(df['label'], df['predicted_sentiment'], average='macro')

# Erstellen Sie ein DataFrame für die Verwirrungsmatrix
confusion_df = pd.DataFrame(conf_matrix, columns=["Predicted Class 0", "Predicted Class 1", "Predicted Class 2"], index=["True Class 0", "True Class 1", "True Class 2"])

# Drucken Sie die Verwirrungsmatrix
print("Verwirrungsmatrix:")
print(confusion_df)

# Erstellen Sie ein DataFrame für Precision und Recall
precision_recall_df = pd.DataFrame({"Precision": precision, "Recall": recall}, index=["Class 0", "Class 1", "Class 2"])

# Drucken Sie die Precision und den Recall für jede Klasse
print("\nPrecision und Recall pro Klasse:")
print(precision_recall_df)

# Drucken Sie den gewichteten Durchschnitt von Precision und Recall (Macro-Durchschnitt)
print(f"\nMacro Precision: {macro_precision:.2f}")
print(f"Macro Recall: {macro_recall:.2f}")

Verwirrungsmatrix:
              Predicted Class 0  Predicted Class 1  Predicted Class 2
True Class 0                275                 27                 10
True Class 1                185                579                105
True Class 2                 32                160                627

Precision und Recall pro Klasse:
         Precision    Recall
Class 0   0.558943  0.881410
Class 1   0.755875  0.666283
Class 2   0.845013  0.765568

Macro Precision: 0.72
Macro Recall: 0.77


In [11]:
precision_recall_df

,Precision,Recall
Class 0,0.558943,0.881410
Class 1,0.755875,0.666283
Class 2,0.845013,0.765568


In [12]:
confusion_df

,Predicted Class 0,Predicted Class 1,Predicted Class 2
True Class 0,275,27,10
True Class 1,185,579,105
True Class 2,32,160,627
